In [7]:
import torch
from vggt.models.vggt import VGGT
from vggt.utils.load_fn import load_and_preprocess_images

device = "cuda" if torch.cuda.is_available() else "cpu"
# bfloat16 is supported on Ampere GPUs (Compute Capability 8.0+) 
dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16

# Initialize the model and load the pretrained weights.
# This will automatically download the model weights the first time it's run, which may take a while.
model = VGGT()
model.load_state_dict(torch.load("weights/model.pt"))
model.to(device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 

In [5]:
import os
image_folder='examples/kitchen/images'
image_names=[]
for image in os.listdir(image_folder):
    image_names.append(os.path.join(image_folder, image))

In [6]:
# Load and preprocess example images (replace with your own image paths)
# image_names = ["path/to/imageA.png", "path/to/imageB.png", "path/to/imageC.png"]  
images = load_and_preprocess_images(image_names).to(device)

with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=dtype):
        # Predict attributes including cameras, depth maps, and point maps.
        predictions = model(images)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!